# Inicializacia

Podme si vyskusat, aky bude mat dopad pouzitie vhodnej inicializacie na nase ukazkove modely.

In [1]:
import numpy as np

from backstage.utils import dataset_Circles, dataset_Flower, draw_DecisionBoundary, draw_TrainingResults
from backstage.layer import Input, Dense, DropOut
from backstage.loss import CreateLossFunction
from backstage.optimizer import GradientDescent
from backstage.model import Model

## 1. Implementacia inicializatora

Zadeklarujme si BaseClass pre inicializator.

In [3]:
class Initializer:
    def __init__(self):
        pass

    def __call__(self, nin, nout):
        pass

Skusme zakladne inicializatory pre Uniform a Normal rozdelenia.

In [4]:
class Normal(Initializer):
    def __init__(self, mean=0.0, stddev=1.0):
        self.mean = mean
        self.stddev = stddev

    def __call__(self, nin, nout):
        # Spravime nahodny tensor s rozmermi (Nout x Nin)
        t = np.random.randn(nout, nin)

        # Posunieme Mean a deviation
        t = (t * self.stddev) + t.mean
        return t

A skusme teraz pouzit inicializator v nasom frameworku

In [6]:
class Dense(Dense):
    def __init__(self, nunits, act='linear', weightsInitializer=Normal):
        super().__init__(act)        
        self.nunits = nunits                            # Pocet neuronov vrstvy
        self.W = None                                   # Maticu vah zatial nemame
        self.b = None                                   # Bias zatial nemame
        self.optimizerContext = None                    # Kontext zatial nemame
        self.weightsInitializer = weightsInitializer    # Odlozime inicializer

    def initialize(self, prevLayer):
        # 1. Inicializacia
        self.shape = (self.nunits, 1)
        self.optimizerContext = None

        # Potom ziskame shape z predchadzajucej vrstvy, aby sme vedeli 
        # urcit rozmer pre maticu vah
        pnx, _ = prevLayer.shape
        nx, _ = self.shape

        # Inicializujeme vahy a bias. Pouzijeme inicializator vah z konstruktoru
        self.W = self.weightsInitializer(nin=pnx, nout=nx)
        self.b = np.zeros((nx, 1), dtype=float)

## 2. Xavier Initializer

Teraz zimplementujeme Xavier (Glorot) Initializer podla slajdu

In [8]:
class XavierNormal(Initializer):
    def __super__(self):
        pass

    def __call__(self, nin, nout):

        # Spocitame pozadovanu varianciu
        variance = 2.0 / (nin + nout)

        # Spravime nahodny tensor s rozmermi (Nout x Nin) a prenasobime scale factorom
        t = np.random.randn(nout, nin) * np.sqrt(variance)
        return t

## 3. He Initializer

A pre uplnost este aj He Initializer

In [9]:
class HeNormal(Initializer):
    def __super__(self):
        pass

    def __call__(self, nin, nout):

        # Spocitame pozadovanu varianciu
        variance = 4.0 / (nin + nout)

        # Spravime nahodny tensor s rozmermi (Nout x Nin) a prenasobime scale factorom
        t = np.random.randn(nout, nin) * np.sqrt(variance)
        return t